In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-15 10:42:45--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  2.83MB/s    in 0.4s    

2024-06-15 10:42:46 (2.83 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



In [2]:
#read to inspect
with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters",len(text))

length of dataset in characters 1115394


In [4]:
#rirst 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
#unique characters in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('vocab size:', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos ={i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s] ##encoder
decode = lambda l: ''.join([itos[i] for i in l]) ##decoder

In [7]:
print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [8]:
#encoding entire dataset
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)

torch.Size([1115394]) torch.int64


In [9]:
#train val split
n = int(0.9*len(data))
train_data, val_data = data[:n], data[n:]

In [10]:
block_size = 8
train_data = train_data[:block_size+1]

In [11]:
x= train_data[:block_size]
y= train_data[1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f"when context {context} target is {target}")

when context tensor([18]) target is 47
when context tensor([18, 47]) target is 56
when context tensor([18, 47, 56]) target is 57
when context tensor([18, 47, 56, 57]) target is 58
when context tensor([18, 47, 56, 57, 58]) target is 1
when context tensor([18, 47, 56, 57, 58,  1]) target is 15
when context tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when context tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [12]:
torch.manual_seed(1337)
batch_size = 4 
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y= torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb= get_batch('train')
print('inputs')
print(xb)
print(xb.shape)
print('targets')
print(yb)
print(yb.shape)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when context {context.tolist()} target is {target}")

inputs
tensor([[18, 47, 56, 57, 58,  1, 15, 47],
        [18, 47, 56, 57, 58,  1, 15, 47],
        [18, 47, 56, 57, 58,  1, 15, 47],
        [18, 47, 56, 57, 58,  1, 15, 47]])
torch.Size([4, 8])
targets
tensor([[47, 56, 57, 58,  1, 15, 47, 58],
        [47, 56, 57, 58,  1, 15, 47, 58],
        [47, 56, 57, 58,  1, 15, 47, 58],
        [47, 56, 57, 58,  1, 15, 47, 58]])
torch.Size([4, 8])
when context [18] target is 47
when context [18, 47] target is 56
when context [18, 47, 56] target is 57
when context [18, 47, 56, 57] target is 58
when context [18, 47, 56, 57, 58] target is 1
when context [18, 47, 56, 57, 58, 1] target is 15
when context [18, 47, 56, 57, 58, 1, 15] target is 47
when context [18, 47, 56, 57, 58, 1, 15, 47] target is 58
when context [18] target is 47
when context [18, 47] target is 56
when context [18, 47, 56] target is 57
when context [18, 47, 56, 57] target is 58
when context [18, 47, 56, 57, 58] target is 1
when context [18, 47, 56, 57, 58, 1] target is 15
when cont

In [13]:
print(xb)

tensor([[18, 47, 56, 57, 58,  1, 15, 47],
        [18, 47, 56, 57, 58,  1, 15, 47],
        [18, 47, 56, 57, 58,  1, 15, 47],
        [18, 47, 56, 57, 58,  1, 15, 47]])


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        #each token direclty reads off the logits for the next token from a lookup table 
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self,idx,targets =None):
        #idx and targets are both [block_size,batch_size]
        logits = self.token_embedding_table(idx)  # [block_size,batch_size,vocab_size] (B,T,C)
        if targets is None:
            loss=None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)

            loss = F.cross_entropy(logits,targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    

m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.4032, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [15]:
#create an optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr = 1e-3)

In [16]:
batch_size = 32
for steps in range(10000):
    #sample a batch of data
    xb,yb = get_batch('train')

    #evalute the loss
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

0.1807720959186554


In [17]:
print(decode(m.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))


pyet Cit Cit Cit Cit Cirst Cit Cit CitKhZXY,rst Cit CiKoduit Cirst Cit Cirst Cirst Cit Cirst Cirst C


### The mathematical trick of self attention

In [18]:
torch.manual_seed(1337)
B,T,C = 4,8,2 #batch, time, channels
x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 2])

In [19]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev =x[b, :t+1]  #(t,C)
        xbow[b,t] = torch.mean(xprev,dim=0) # (C)

In [20]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [21]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [22]:
# version : 2
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(dim=1,keepdim=True)
xbow2 = wei @ x # (T,T) @ (B,T,C) ----> (B,T,T) @ (B,T,C) ----> (B,T,C)
torch.allclose(xbow,xbow2)

False

In [23]:
xbow[0],xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [24]:
# version:3 , Softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
print("Before softmax \n",wei)
wei = F.softmax(wei, dim = -1)
xbow3 = wei @ x

Before softmax 
 tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [25]:
print(wei)
print(xbow3[0])


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [26]:
torch.tril(a = torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [27]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a/torch.sum(a, dim=0, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print("a : ")
print(a)
print("b : ")
print(b)
print("c : ")
print(c)

a : 
tensor([[0.3333, 0.0000, 0.0000],
        [0.3333, 0.5000, 0.0000],
        [0.3333, 0.5000, 1.0000]])
b : 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c : 
tensor([[0.6667, 2.3333],
        [3.6667, 4.3333],
        [9.6667, 9.3333]])


In [28]:
nn.Embedding(10,3)

Embedding(10, 3)

In [33]:
# version:4 , self attention
B,T,C = 4,8,32
x =torch.randn(B,T,C)

#implementing single head self attension
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias= False)
value = nn.Linear(C, head_size, bias=False)
k= key(x)
q = query(x)

wei = q @ k.transpose(-2,-1)  # (B,T, 16) @ (B,16,T) ---> (B,T,T)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
print("Before softmax \n",wei)
wei = F.softmax(wei, dim = -1)

v = value(x)
out = wei @ v

out.shape

Before softmax 
 tensor([[[ 8.8887e-01,        -inf,        -inf,        -inf,        -inf,
                 -inf,        -inf,        -inf],
         [-1.1939e+00, -2.5734e-03,        -inf,        -inf,        -inf,
                 -inf,        -inf,        -inf],
         [ 6.2490e-01, -9.9382e-01,  7.1121e-01,        -inf,        -inf,
                 -inf,        -inf,        -inf],
         [-3.6508e+00, -1.5804e+00, -1.4319e+00, -9.4028e-01,        -inf,
                 -inf,        -inf,        -inf],
         [-1.3706e+00,  8.1821e-01, -2.5999e+00, -8.9379e-02,  1.2676e+00,
                 -inf,        -inf,        -inf],
         [-6.8650e-01, -5.1722e-01,  3.0751e-01, -2.6841e+00, -2.0892e-01,
          -1.0684e+00,        -inf,        -inf],
         [-3.2220e+00, -6.2742e-02,  1.1689e+00,  1.7562e+00, -1.2834e-01,
          -2.6596e-01,  3.6047e+00,        -inf],
         [ 1.5367e+00,  1.4740e+00, -4.7280e-01, -9.1231e-01,  1.1701e-01,
          -1.4669e-01,  3.1146e+0

torch.Size([4, 8, 16])

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [34]:
q = torch.randn(B,T,head_size)
k= torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) * head_size**-0.5

In [35]:
k.var()


tensor(0.9752)

In [36]:
q.var()


tensor(1.0193)

In [37]:
wei.var()

tensor(1.0127)